<a href="https://colab.research.google.com/github/venkat2ram/ML_From_Scratch/blob/master/NeuralNet_From_Scratch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
data=pd.read_csv('/content/data_banknote_authentication.csv')

In [441]:
data.head()

,a,b,c,d,y
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [0]:
def init_params(layer_dims):
  np.random.seed(3)
  L=len(layer_dims)
  params={}
  for l in range(1,L):
    #params['w'+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])/0.01
    params['w'+str(l)]=np.ones((layer_dims[l],layer_dims[l-1]))
    params['b'+str(l)]=np.zeros((layer_dim[l],1))
  return params


In [0]:
def sigmoid(Z):
  A=1/(1+np.exp(np.dot(-1,Z)))
  cache=(Z)
  return A,cache

In [0]:
def forward_prop(X,params):
  A=X
  caches=[]
  L=len(params)//2
  for l in range(1,L+1):
    A_prev=A
    Z=np.dot(params['w'+str(l)],A_prev)+params['b'+str(l)]
    linear_cache=(A_prev,params['w'+str(l)],params['b'+str(l)])
    A,activation_cache=sigmoid(Z)
    cache=(linear_cache,activation_cache)
    caches.append(cache)
  return A,caches   


In [0]:
def cost_function(A, Y):
  m=Y.shape[1]
  cost=(-1/m)*np.sum(np.dot(np.log(A),Y.T)+np.dot(np.log(1-A),1-Y.T))
  return cost

In [0]:
def one_layer_backward(dA,cache):
  linear_cache, activation_cache=cache
  
  Z=activation_cache
  sig,c=sigmoid(Z)
  dZ=dA*sig
  #*(np.ones(Z.shape)-sig)

  A_prev,W,b=linear_cache
  m=A_prev.shape[1]
  dW=(1/m)*(np.dot(dZ,A_prev.T))
  dB=(1/m)*(np.sum(dZ,axis=1,keepdims=True))
  dA_prev=(1/m)*np.dot(W.T,dZ)

  return dA_prev,dW,dB

In [0]:
def backprop(AL,Y,caches):
  grads={}
  L=len(caches)
  Y = Y.reshape(AL.shape)
  dA=-1*(np.divide(Y,AL)+np.divide(1-Y,1-AL))
  grads['dA'+str(L-1)],grads['dW'+str(L-1)],grads['dB'+str(L-1)]=one_layer_backward(dA,caches[L-1])
  for l in reversed(range(L-1)):
    grads['dA'+str(l)],grads['dW'+str(l)],grads['dB'+str(l)]=one_layer_backward(grads['dA'+str(l+1)],cache[l])
  return grads


In [0]:
def update_parameters(parameters, grads, learning_rate):
  L=len(parameters)//2
  for l in range(1,L):
    parameters['w'+str(l)]=parameters['w'+str(l)]-learning_rate*grads['dW'+str(l-1)]
    parameters['b'+str(l)]=parameters['b'+str(l)]-learning_rate*grads['dB'+str(l-1)]
  return parameters


In [0]:
def train(X, Y, layer_dims, epochs, lr):
    params = init_params(layer_dims)
    cost_history = []
    
    for i in range(epochs):
        Y_hat, caches = forward_prop(X, params)
        cost = cost_function(Y_hat, Y)
        cost_history.append(cost)
        grads = backprop(Y_hat, Y, caches)
        
        params = update_parameters(params, grads, lr)
        if(i%100000==1):
          print(cost)
        
        
    return params, cost_history

In [434]:
p,c=train(a,Y,layer_dim,1000000,1)

31.38856288237458
30.494132607443618
30.49428637918958
30.494440108779333
30.494593795003453
30.494747437861378


KeyboardInterrupt: ignored

**Rough**

In [416]:
a=np.array([[0.2,0.2,0.3],[0.3,0.4,0.5],[0.4,0.3,0.5]])
b=np.array([[0.3,0.4,0.5],[0.6,0.7,0.8]])
Y=np.array([[0,1,1],[1,0,0],[0,1,1],[1,0,0]])
cost_function(a,b)


4.742705654642286

In [0]:
layer_dim=[3,4,4,4]

In [0]:
params=init_params(layer_dim)

In [0]:
y_hat,caches=forward_prop(a,params)

In [420]:
grads=backprop(y_hat,Y,caches)

5
4
1


ValueError: ignored

In [0]:
grads['dA'+str(2)],grads['dW'+str(2)],grads['dB'+str(2)]=one_layer_backward(dA,caches[2])

In [0]:
grads['dA2'].reshape(1,-1)

In [422]:
grads['dA2']

array([-1.0267612e+70, -1.0267612e+70, -1.0267612e+70, -1.0267612e+70])

In [0]:
caches

In [0]:
dA1=np.array([[ 2.40363348e+08,  2.40363219e+08,  2.61996268e+08],
        [ 2.15179835e+07,  2.15175481e+07,  2.34541597e+07],
        [-3.31390135e+08, -3.31390064e+08, -3.61215665e+08],
        [ 5.11541584e+07,  5.11539908e+07,  5.57579265e+07]])

In [0]:
Z=np.array([[0.94499721, 0.94499721, 0.95864549],
          [0.94499721, 0.94499721, 0.95864549],
          [0.94499721, 0.94499721, 0.95864549],
          [0.94499721, 0.94499721, 0.95864549]])

In [0]:
sig,xs=sigmoid(Z)
dZ=dA*sig*(np.ones(Z.shape)-sig)

In [0]:
dZ

In [0]:
grads['dA4']

In [0]:
len(caches)

In [0]:
dA=-1*(np.divide(Y,y_hat)+np.divide(1-Y,1-y_hat))

In [0]:
dA

In [0]:
np.sum(dA,axis=1,keepdims=True)

In [0]:
grads={}

In [0]:
grads['dA'+str(2)],grads['dW'+str(2)],grads['dB'+str(2)]=one_layer_backward(dA,caches[2])

In [0]:
grads['dB'+str(0)]

In [0]:
np.sum(grads['dB'+str(0)],axis=2)

In [0]:
grads['dA'+str(1)],grads['dW'+str(1)],grads['dB'+str(1)]=one_layer_backward(grads['dA'+str(2)],cache[l])

In [0]:
caches

In [0]:
y_hat

In [0]:
dA=-1*np.divide(b,y_hat)+np.divide(1-b,1-y_hat)

In [0]:
one_layer_backward(dA,cache[2])

In [0]:
params

In [0]:
cache

In [0]:
params

In [0]:
grads=

In [0]:
np.dot(params['w1'],a)

In [0]:
sigmoid(2)

In [0]:
c1,c2=cache[2]

In [0]:
c2

In [0]:
dA

In [0]:
dA*sigmoid(c2)*sigmoid(1-c2)

In [0]:
d1*sigmoid(d2)*(np.ones(d2.shape)-sigmoid(d2))

In [0]:
sigmoid(d2)

In [0]:
np.ones(d2.shape)-sigmoid(d2)

In [0]:
len(caches)

In [0]:
a1=np.array([[[1,2],[3,4]],[[1,2],[3,4]]])

In [0]:
np.sum(a1,axis=0)

In [0]:
ax=np.array([[[1.82966561e+71, 5.37101649e+34, 3.10119951e+31],
         [1.82966561e+71, 5.37101649e+34, 3.10119951e+31],
         [1.82966561e+71, 5.37101649e+34, 3.10119951e+31],
         [1.82966561e+71, 5.37101649e+34, 3.10119951e+31]],
 
        [[3.14106226e+80, 3.07004636e+50, 7.37319456e+50],
         [3.14106226e+80, 3.07004636e+50, 7.37319456e+50],
         [3.14106226e+80, 3.07004636e+50, 7.37319456e+50],
         [3.14106226e+80, 3.07004636e+50, 7.37319456e+50]]])

In [0]:
np.sum(a3,axis=0)

In [0]:
axx=np.array([[[ 1,2,3]],
 
        [[1,2,3]]])

In [0]:
params